In [1]:
import multiprocessing
multiprocessing.cpu_count()

8

In [30]:
from multiprocessing import Process, Queue, Pipe, Manager
import os, sys, time
from os import getpid
from random import randint
from time import time, sleep

# 进程

## Multiprocessing
Process进程对象的守护进程是通过设置daemon属性来完成的。

In [50]:
def download_task(filename):
    print('启动下载进程，进程号[%d].' % getpid())
    print('开始下载%s...' % filename)
    time_to_download = randint(5, 10)
    sleep(time_to_download)
    print('%s下载完成! 耗费了%d秒' % (filename, time_to_download))


def main():
    start = time()
    p1 = Process(target = download_task, args = ('Python从入门到住院.pdf', ))
    p1.start()
    p2 = Process(target=download_task, args=('Peking Hot.avi', ))
    p2.start()
    p1.join()
    p2.join()
    end = time()
    print('总共耗费了%.2f秒.' % (end - start))

if __name__ == '__main__':
    main()

启动下载进程，进程号[78542].
启动下载进程，进程号[78543].
开始下载Python从入门到住院.pdf...
开始下载Peking Hot.avi...
Python从入门到住院.pdf下载完成! 耗费了6秒
Peking Hot.avi下载完成! 耗费了8秒
总共耗费了8.03秒.


构造方法：  
Process([group [, target [, name [, args [, kwargs]]]]])  
group: 线程组   
target: 要执行的方法  
name: 进程名  
args/kwargs: 要传入方法的参数  
  
实例方法：  
is_alive()：返回进程是否在运行,bool类型。  
join([timeout])：阻塞当前上下文环境的进程程，直到调用此方法的进程终止或到达指定的timeout（可选参数）。  
start()：进程准备就绪，等待CPU调度  
run()：strat()调用run方法，如果实例进程时未制定传入target，这star执行t默认run()方法。  
terminate()：不管任务是否完成，立即停止工作进程  
  
属性：  
daemon：和线程的setDeamon功能一样  
name：进程名字  
pid：进程号


## Fork

In [24]:
import os
print("Process (%s) start..." % os.getpid())    #获取当前进程的id
pid = os.fork()
if pid == 0:
    print("子进程为 (%s) ，主进程为 %s" % (os.getpid(), os.getppid()))
else:
    print('我 (%s) 创建了子进程 (%s).' % (os.getpid(), pid))

Process (88293) start...
我 (88293) 创建了子进程 (89135).
子进程为 (89135) ，主进程为 88293


## Pool

In [32]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('启动下载进程，进程号[%d].' % getpid())
    print('开始下载%s...' % name)
    start = time.time()
    time.sleep(random.random() * 2)
    end = time.time()
    print('%s 执行时长为 %0.2f seconds.' % (name, (end - start)))
    
if __name__=='__main__':
    print("主进程 %s." % os.getpid())
    p = Pool(8)    #pool默认大小是CPU核数
    for i in range(8):
        p.apply_async(long_time_task, args = ("任务" + str(i), ))
    print("等待所有子进程执行")
    p.close()
    p.join()           
    '''对Pool对象调用join()方法会等待所有子进程执行完毕，
       调用join()之前必须先调用close()，调用close()之后就不能继续添加新的Process了。'''
    print("执行完毕")

主进程 88293.
启动下载进程，进程号[89209].
启动下载进程，进程号[89210].
启动下载进程，进程号[89212].
启动下载进程，进程号[89214].
启动下载进程，进程号[89211].
启动下载进程，进程号[89213].
开始下载任务3...
启动下载进程，进程号[89215].
开始下载任务1...
开始下载任务0...
启动下载进程，进程号[89216].
开始下载任务2...
开始下载任务4...
开始下载任务6...
开始下载任务5...
开始下载任务7...
任务5 执行时长为 0.07 seconds.
等待所有子进程执行
任务0 执行时长为 0.21 seconds.
任务7 执行时长为 0.33 seconds.
任务3 执行时长为 0.35 seconds.
任务1 执行时长为 0.41 seconds.
任务4 执行时长为 0.74 seconds.
任务6 执行时长为 0.77 seconds.
任务2 执行时长为 1.14 seconds.
执行完毕


## Queue
Queue在多线程中也说到过，在生成者消费者模式中使用，是线程安全的，是生产者和消费者中间的数据管道，那在python多进程中，它其实就是进程之间的数据管道，实现进程通信。

In [33]:
def fun1(q,i):
    print('子进程%s 开始put数据' %i)
    q.put('我是%s 通过Queue通信' %i)

if __name__ == '__main__':
    q = Queue()

    process_list = []
    for i in range(3):
        p = Process(target=fun1,args=(q,i,))  #注意args里面要把q对象传给我们要执行的方法，这样子进程才能和主进程用Queue来通信
        p.start()
        process_list.append(p)

    for i in process_list:
        p.join()

    print('主进程获取Queue数据')
    print(q.get())
    print(q.get())
    print(q.get())
    print('结束测试')

子进程0 开始put数据
子进程1 开始put数据
子进程2 开始put数据
主进程获取Queue数据
我是0 通过Queue通信
我是1 通过Queue通信
我是2 通过Queue通信
结束测试


## Pipe
管道Pipe和Queue的作用大致差不多，也是实现进程间的通信

In [34]:
def fun1(conn):
    print('子进程发送消息：')
    conn.send('你好主进程')
    print('子进程接受消息：')
    print(conn.recv())
    conn.close()

if __name__ == '__main__':
    conn1, conn2 = Pipe() #关键点，pipe实例化生成一个双向管
    p = Process(target=fun1, args=(conn2,)) #conn2传给子进程
    p.start()
    print('主进程接受消息：')
    print(conn1.recv())
    print('主进程发送消息：')
    conn1.send("你好子进程")
    p.join()
    print('结束测试')

子进程发送消息：
子进程接受消息：
你好子进程
主进程接受消息：
你好主进程
主进程发送消息：
结束测试


## Manager
实现数据共享，即一个进程去更改另一个进程的数据

In [78]:
def fun1(dic,lis,index):

    dic[index] = 'a'
    dic['2'] = 'b'    
    lis.append(index)    #[0,1,2,3,4,0,1,2,3,4,5,6,7,8,9]
    #print(l)

if __name__ == '__main__':
    with Manager() as manager:
        dic = manager.dict()#注意字典的声明方式，不能直接通过{}来定义
        l = manager.list(range(5))#[0,1,2,3,4]

        process_list = []
        for i in range(10):
            p = Process(target=fun1, args=(dic,l,i))
            p.start()
            process_list.append(p)

        for res in process_list:
            res.join()
        print(dic)
        print(l)

{0: 'a', '2': 'b', 1: 'a', 2: 'a', 3: 'a', 4: 'a', 5: 'a', 6: 'a', 7: 'a', 8: 'a', 9: 'a'}
[0, 1, 2, 3, 4, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


对`Pool`对象调用`join()`方法会等待所有子进程执行完毕，调用`join()`之前必须先调用`close()`，调用`close()`之后就不能继续添加新的`Process`了

In [35]:
def fun1(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

if __name__=='__main__':
    pool = Pool(5) #创建一个5个进程的进程池

    for i in range(10):
        pool.apply_async(func=fun1, args=(i,))

    pool.close()
    pool.join()
    print('结束测试')


Run task 2 (89339)...
Run task 0 (89337)...
Run task 3 (89340)...
Run task 1 (89338)...
Run task 4 (89341)...
Task 0 runs 0.75 seconds.
Run task 5 (89337)...
Task 1 runs 1.10 seconds.
Run task 6 (89338)...
Task 2 runs 1.17 seconds.
Run task 7 (89339)...
Task 3 runs 1.36 seconds.
Run task 8 (89340)...
Task 5 runs 1.03 seconds.
Run task 9 (89337)...
Task 4 runs 2.37 seconds.
Task 8 runs 1.38 seconds.
Task 9 runs 1.70 seconds.
Task 6 runs 2.49 seconds.
Task 7 runs 2.63 seconds.
结束测试


## Subprocess

In [68]:
import subprocess

print("$ nslookup www.python.org")
r = subprocess.call(['nslookup', 'www.python.org'])
print("Exit code:", r)

$ nslookup www.python.org
Exit code: 0


In [69]:
print('$ nslookup')
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

$ nslookup
Server:		222.200.115.251
Address:	222.200.115.251#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:
python.org	nameserver = ns4.p11.dynect.net.
python.org	nameserver = ns1.p11.dynect.net.
python.org	nameserver = ns2.p11.dynect.net.
python.org	nameserver = ns3.p11.dynect.net.
mail.python.org	internet address = 188.166.95.178
ns1.p11.dynect.net	internet address = 208.78.70.11
ns2.p11.dynect.net	internet address = 204.13.250.11
ns3.p11.dynect.net	internet address = 208.78.71.11
ns4.p11.dynect.net	internet address = 204.13.251.11
mail.python.org	has AAAA address 2a03:b0c0:2:d0::71:1


Exit code: 0


# 线程(不适用计算密集型)
![线程](pic/Threading.jpeg)

In [1]:
import time, threading
from queue import Queue

In [4]:
def say(name):
    print('你好%s at %s\n' %(name,time.ctime()))
    time.sleep(2)
    print("结束%s at %s\n" %(name,time.ctime()))

def listen(name):
    print('你好啊%s at %s' % (name,time.ctime()))
    time.sleep(4)
    print("结束咯%s at %s" % (name,time.ctime()))

if __name__ == '__main__':
    t1 = threading.Thread(target=say,args=('tony',))  #Thread是一个类，实例化产生t1对象，这里就是创建了一个线程对象t1
    t1.start() #线程执行
    t2 = threading.Thread(target=listen, args=('simon',)) #这里就是创建了一个线程对象t2
    t2.start()

    print("\n程序结束=====================")

你好tony at Sun Dec 15 10:44:28 2019

你好啊simon at Sun Dec 15 10:44:28 2019

程序结束=====================
结束tony at Sun Dec 15 10:44:30 2019

结束咯simon at Sun Dec 15 10:44:32 2019


## join
如果子线程不加join，则主线程也会执行打印，但是主线程不会结束，还是需要待非守护子线程结束之后，主线程才结束

In [5]:
if __name__ == '__main__':
    t1 = threading.Thread(target=say,args=('tony',))  #Thread是一个类，实例化产生t1对象，这里就是创建了一个线程对象t1
    t1.start() #线程执行
    t2 = threading.Thread(target=listen, args=('simon',)) #这里就是创建了一个线程对象t2
    t2.start()

    t1.join() #join等t1子线程结束，主线程打印并且结束
    t2.join() #join等t2子线程结束，主线程打印并且结束
    print("程序结束=====================")

你好tony at Sun Dec 15 10:46:18 2019
你好啊simon at Sun Dec 15 10:46:18 2019

结束tony at Sun Dec 15 10:46:20 2019

结束咯simon at Sun Dec 15 10:46:22 2019
程序结束=====================


## 守护线程
如果某个子线程设置为守护线程，主线程其实就不用管这个子线程了，当所有其他非守护线程结束，主线程就会退出，而守护线程将和主线程一起退出，守护主线程，这就是守护线程的意思

In [6]:
if __name__ == '__main__':
    t1 = threading.Thread(target=say,args=('tony',))  #Thread是一个类，实例化产生t1对象，这里就是创建了一个线程对象t1
    t1.setDaemon(True)
    t1.start() #线程执行
    t2 = threading.Thread(target=listen, args=('simon',)) #这里就是创建了一个线程对象t2
    t2.start()

    print("\n程序结束=====================")

你好tony at Sun Dec 15 10:51:58 2019

你好啊simon at Sun Dec 15 10:51:58 2019
程序结束=====================

结束tony at Sun Dec 15 10:52:00 2019

结束咯simon at Sun Dec 15 10:52:02 2019


In [8]:
if __name__ == '__main__':
    t1 = threading.Thread(target=say,args=('tony',))  #Thread是一个类，实例化产生t1对象，这里就是创建了一个线程对象t1
    t1.start() #线程执行
    t2 = threading.Thread(target=listen, args=('simon',)) #这里就是创建了一个线程对象t2
    t2.setDaemon(True)
    t2.start()
    print("\n程序结束=====================")

你好tony at Sun Dec 15 10:53:19 2019

你好啊simon at Sun Dec 15 10:53:19 2019
程序结束=====================

结束tony at Sun Dec 15 10:53:21 2019

结束咯simon at Sun Dec 15 10:53:23 2019


In [8]:
# 新线程执行的代码:
def loop():
    print("thread %s is running..." % threading.current_thread().name)
    n = 0
    while n < 5:
        n += 1
        print("thread %s >>> %s" % (threading.current_thread().name, n))
        time.sleep(1)
    print("thread %s ended." % threading.current_thread().name)
    
print("thread %s is running..." % threading.current_thread().name)
t = threading.Thread(target= loop, name = "LoopThread")
t.start()
t.join()
print("thread %s ended." % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


## 锁机制
![锁](pic/locking.jpeg)

In [ ]:
num = 100

def fun_sub():
    global num
    # num -= 1
    print('num=', num)
    num2 = num
    time.sleep(0.001)
    num = num2-1
    print('num=', num)

if __name__ == '__main__':
    print('开始测试同步锁 at %s' % time.ctime())

    thread_list = []
    for thread in range(100):
        t = threading.Thread(target=fun_sub)
        t.start()
        thread_list.append(t)

    for t in thread_list:
        t.join()
    print('num is %d' % num)
    print('结束测试同步锁 at %s' % time.ctime())

- 因为GIL，只有一个线程（假设线程1）拿到了num这个资源，然后把变量赋值给num2,sleep 0.001秒，这时候num=100
- 当第一个线程sleep 0.001秒这个期间，这个线程会做yield操作，就是把cpu切换给别的线程执行（假设线程2拿到个GIL，获得cpu使用权），线程2也和线程1一样也拿到num,返回赋值给num2，然sleep,这时候，其实num还是=100.
- 线程2 sleep时候，又要yield操作，假设线程3拿到num,执行上面的操作，其实num有可能还是100
- 等到后面cpu重新切换给线程1，线程2，线程3上执行的时候，他们执行减1操作后，其实等到的num其实都是99，而不是顺序递减的。
- 其他剩余的线程操作如上

In [ ]:
num = 100
def fun_sub():
    global num
    lock.acquire()
    print('----加锁----')
    print('现在操作共享资源的线程名字是:',t.name)
    print('num=', num)
    num2 = num
    time.sleep(0.001)
    num = num2-1
    lock.release()
    print('----释放锁----')
    print('num=', num)

if __name__ == '__main__':
    print('开始测试同步锁 at %s' % time.ctime())

    lock = threading.Lock() #创建一把同步锁

    thread_list = []
    for thread in range(100):
        t = threading.Thread(target=fun_sub)
        t.start()
        thread_list.append(t)

    for t in thread_list:
        t.join()
    print('num is %d' % num)
    print('结束测试同步锁 at %s' % time.ctime())

### 死锁

In [21]:
lock_apple = threading.Lock()
lock_banana = threading.Lock()

class MyThread(threading.Thread):

    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        self.fun1()
        self.fun2()

    def fun1(self):

        lock_apple.acquire()  # 如果锁被占用,则阻塞在这里,等待锁的释放

        print ("线程 %s , 想拿: %s--%s" %(self.name, "苹果",time.ctime()))

        lock_banana.acquire()
        print ("线程 %s , 想拿: %s--%s" %(self.name, "香蕉",time.ctime()))
        lock_banana.release()
        lock_apple.release()


    def fun2(self):

        lock_banana.acquire()
        print ("线程 %s , 想拿: %s--%s" %(self.name, "香蕉",time.ctime()))
        time.sleep(0.1)

        lock_apple.acquire()
        print ("线程 %s , 想拿: %s--%s" %(self.name, "苹果",time.ctime()))
        lock_apple.release()

        lock_banana.release()

if __name__ == "__main__":
    for i in range(0, 10):  #建立10个线程
        my_thread = MyThread()  #类继承法是python多线程的另外一种实现方式
        my_thread.start()

线程 Thread-1120 , 想拿: 苹果--Sun Dec 15 11:04:25 2019
线程 Thread-1120 , 想拿: 香蕉--Sun Dec 15 11:04:25 2019
线程 Thread-1120 , 想拿: 香蕉--Sun Dec 15 11:04:25 2019
线程 Thread-1121 , 想拿: 苹果--Sun Dec 15 11:04:25 2019


#### RLock

In [22]:
lock = threading.RLock()  #递归锁


class MyThread(threading.Thread):

    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        self.fun1()
        self.fun2()

    def fun1(self):

        lock.acquire()  # 如果锁被占用,则阻塞在这里,等待锁的释放

        print ("线程 %s , 想拿: %s--%s" %(self.name, "苹果",time.ctime()))

        lock.acquire()
        print ("线程 %s , 想拿: %s--%s" %(self.name, "香蕉",time.ctime()))
        lock.release()
        lock.release()


    def fun2(self):

        lock.acquire()
        print ("线程 %s , 想拿: %s--%s" %(self.name, "香蕉",time.ctime()))
        time.sleep(0.1)

        lock.acquire()
        print ("线程 %s , 想拿: %s--%s" %(self.name, "苹果",time.ctime()))
        lock.release()

        lock.release()

if __name__ == "__main__":
    for i in range(0, 10):  #建立10个线程
        my_thread = MyThread()  #类继承法是python多线程的另外一种实现方式
        my_thread.start()

线程 Thread-1130 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1130 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1130 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1130 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1131 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1131 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1131 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1131 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1133 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1133 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1133 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1133 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1135 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1135 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1135 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1135 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1137 , 想拿: 苹果--Sun Dec 15 11:05:15 2019
线程 Thread-1137 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1137 , 想拿: 香蕉--Sun Dec 15 11:05:15 2019
线程 Thread-1137 , 想拿: 苹果--Sun Dec 15 11:05:15 2019


### ThreadLocal

In [2]:
import threading

'''创建全局ThreadLocal对象'''
local_school = threading.local()

def process_student():
    '''获取当前线程关联的student'''
    std = local_school.student
    print("Hello, %s (in %s)" % (std, threading.current_thread().name))
    
def process_thread(name):
    '''绑定ThreadLocal的的student'''
    local_school.student = name
    process_student()
    
t1 = threading.Thread(target = process_thread, args = ("Alice",), name = "Thread-A")
t2 = threading.Thread(target = process_thread, args = ("Bob",), name = "Thread-B")
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)
Hello, Bob (in Thread-B)


#### 全局变量`local_school`就是一个`ThreadLocal`对象，每个`Thread`对它都可以读写`student`属性，但互不影响。你可以把`local_school`看成全局变量，但每个属性如`local_school.student`都是线程的局部变量，可以任意读写而互不干扰，也不用管理锁的问题，`ThreadLocal`内部会处理。

#### 可以理解为全局变量`local_school`是一个`dict`，不但可以用`local_school.student`，还可以绑定其他变量，如`local_school.teacher`等等。

#### `ThreadLocal`最常用的地方就是为每个线程绑定一个数据库连接，`HTTP`请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。

In [1]:
import _thread, time
# 定义线程函数
def print_time(threadName, delay):
    count = 0
    while count < 5:
        time.sleep(delay)
        count += 1
        # 返回当前时间的时间戳（1970纪元后经过的浮点秒数）, 并格式化输出
        print("{}: {}".format(threadName, time.ctime(time.time()) ))
try:
    _thread.start_new_thread( print_time, ("Thread-1", 2))
    _thread.start_new_thread( print_time, ("Thread-2", 4))
except:
    print("Error")

while 1:
    # 让线程有足够的时间完成
    pass

Thread-1: Sun Mar 17 14:40:29 2019
Thread-2: Sun Mar 17 14:40:31 2019
Thread-1: Sun Mar 17 14:40:31 2019
Thread-1: Sun Mar 17 14:40:33 2019
Thread-2: Sun Mar 17 14:40:35 2019
Thread-1: Sun Mar 17 14:40:35 2019
Thread-1: Sun Mar 17 14:40:37 2019
Thread-2: Sun Mar 17 14:40:39 2019
Thread-2: Sun Mar 17 14:40:43 2019
Thread-2: Sun Mar 17 14:40:47 2019


KeyboardInterrupt: 

# 协程

In [8]:
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.get_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

RuntimeError: This event loop is already running

Hello world!
Hello again!


In [6]:
@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop = asyncio.get_event_loop()
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

Hello world! (<_MainThread(MainThread, started 4571387328)>)
Hello world! (<_MainThread(MainThread, started 4571387328)>)
Hello again! (<_MainThread(MainThread, started 4571387328)>)
Hello again! (<_MainThread(MainThread, started 4571387328)>)


In [7]:
@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

loop = asyncio.get_event_loop()
tasks = [wget(host) for host in ['www.sina.com.cn', 'www.sohu.com', 'www.163.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: This event loop is already running

wget www.163.com...
wget www.sohu.com...
wget www.sina.com.cn...
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Thu, 16 May 2019 16:53:11 GMT
www.sohu.com header > Cache-Control: max-age=60
www.sohu.com header > X-From-Sohu: X-SRC-Cached
www.sohu.com header > Content-Encoding: gzip
www.sohu.com header > FSS-Cache: HIT from 3963534.5929624.5300396
www.sohu.com header > FSS-Proxy: Powered by 2695201.3416107.4009004
www.163.com header > HTTP/1.1 302 Moved Temporarily
www.163.com header > Date: Thu, 16 May 2019 16:54:11 GMT
www.163.com header > Content-Length: 0
www.163.com header > Connection: close
www.163.com header > Server: Cdn Cache Server V2.0
www.163.com header > Location: http://www.163.com/special/0077jt/error_isp.html
www.163.com header > X-Via: 1.0 PSgdzjdx6qe66:3 (Cdn Cache Server V2.0)
www.sina.com.cn header > HTTP/1.1 302 

In [23]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())

asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop